# Execute metagenome taxonomic profile
Jacobo de la Cuesta-Zuluaga. July 2024.

The aim of this notebook is to perform quality control of raw metagenome reads and run taxprofiler pipeline


Note: the human (and other eukaryote) genomes was obtained from https://doi.org/10.5281/zenodo.4629921.

 

## Before we start

The execution of the pipeline requires `conda` to be installed and an environment with `nextflow` available. You can find instructions about how to install conda [here](https://conda.io/projects/conda/en/latest/user-guide/install/index.html).

In addition, we'll be using the `nf-core` pipeline `taxprofiler`. You can read more about `nf-core` [here](https://nf-co.re/) was well as exploring the [pipeline's documentation](https://nf-co.re/taxprofiler/1.1.8/).  

Be sure to add the following to your `~/.bashrc` file. With that, we're specifying a centralized folder where the `nf-core` pipelines will be downloaded. Make sure to modify the path as required
```
export NXF_SINGULARITY_CACHEDIR="/mnt/lustre/groups/maier/YOUR_M3HPC_USERNAME/bin/nf-core"
```


## Load libraries and set paths

First, we'll set up the libraries and the work directory where we'll save our files

In [ ]:
# Libraries
library(tidyverse)
library(conflicted)

In [ ]:
# Solve conflicts
conflicts_prefer(dplyr::filter)

In [ ]:
# Directories
# Base directory
base_dir = "/PATH/TO/YOUR/PROJECT/FOLDER"

# Data
data_dir = file.path(base_dir, "data")
dir.create(data_dir)

# Sequences
seq_dir = file.path(data_dir, "raw_sequences")
dir.create(seq_dir)

# Out
out_dir = file.path(data_dir, "taxprofiler")
dir.create(out_dir)

# sheets dir
sheets_dir = file.path(data_dir, "sheets")
dir.create(sheets_dir)

# Software
conda_env = "nextflow"

## Download test files

For the present example, we'll use publicly available metagenome files. They correspond to multiple sequencing runs of two samples, this means that the same sample was sequenced multiple times to achieve the desired sequencing depth. We'll use these samples to illustrate how to merge multiple sequencing runs to obtain the metagenome profile of a single sample.

In [ ]:
# URL of the files from the ENA
example_fastqs = c("ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880517/MI-142-H.R1.RUN0129.L7.fastq.gz", 
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880518/MI-142-H.R1.RUN0118.L4.fastq.gz",  
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880517/MI-142-H.R2.RUN0129.L7.fastq.gz", 
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880518/MI-142-H.R2.RUN0118.L4.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880579/MI-237-H.R2.RUN0129.L7.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880581/MI-237-H.R1.RUN0118.L2.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880577/MI-237-H.R1.RUN0173.L6.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880582/MI-237-H.R1.RUN0102.L5.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880579/MI-237-H.R1.RUN0129.L7.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880581/MI-237-H.R2.RUN0118.L2.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880577/MI-237-H.R2.RUN0173.L6.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880582/MI-237-H.R2.RUN0102.L5.fastq.gz")

example_fastqs

In [ ]:
# Download files
# This will take a few minutes
#map(example_fastqs, function(url){
#    download.file(url = url, destfile = file.path(seq_dir, basename(url)), method = "wget")
#})

## Create Samples file

We need to tell the pipeline which files correspond to which samples, to which sequencing run they correspond and where in our machine are stored those files. We do this by creating table where we specify the sample name and the location of the forward and reverse `fastq` files. 

In addition, we need to tell the pipeline which sequencing technology we used. In this case, we used an _Illumina_ machine, meaning that we got short reads. The specific software used by the pipeline will change according to this.

**Note** that you can create this table by hand using excel or a text editor program, and exporting it as a `csv` file. In this example we're doing this programatically to use the information of the sample name from the full path of the files.

In [ ]:
# List raw sequences
raw_seq_list = list.files(seq_dir,  
        pattern = "fastq.gz",
        full.names = TRUE)
# F
forward_reads = raw_seq_list %>%
    str_subset("R1")
#R
reverse_reads = raw_seq_list %>%
    str_subset("R2")

In [ ]:
# Combine
reads_tax_df = data.frame(fastq_1 = forward_reads, # Full path of forward reads
        fastq_2 = reverse_reads, # Full path of reverse reads
        instrument_platform = "ILLUMINA", # Sequencing platform 
        fasta = "") %>% # Empty field since we don't have fasta files
    mutate(sample = basename(fastq_1), # Sample name from the file
        sample = str_remove(sample, "\\.R1.*")) %>%
    group_by(sample) %>%
    mutate(run_accession = str_c("run_", row_number())) %>% # If more than one run, specify which run it was
    relocate(sample, instrument_platform, run_accession) # Reorder columns

reads_tax_df %>%
    head()

In [ ]:
# Write samples file
samples_sheet_tax = file.path(sheets_dir, "samples_file_taxprofiler.csv")
write_csv(reads_tax_df,file = samples_sheet_tax)

## Run pipeline

The pipeline requires a file with the location of the databases for the software to be used. 

In [ ]:
# Create dbs file
dbs_df = data.frame(tool = c("kraken2","bracken", "metaphlan", "motus"),
    db_name = c("k2_standard_16gb", "B_standard_16gb", "metaphlan", "db_mOTU"),
    db_params = c("", ";-r 150", "", ""),
    db_path = c("/mnt/lustre/groups/maier/databases/Kraken_Bracken/k2_standard_16gb/k2_standard_16gb_20240605.tar.gz",
                "/mnt/lustre/groups/maier/databases/Kraken_Bracken/k2_standard_16gb/k2_standard_16gb_20240605.tar.gz",
                "/mnt/lustre/groups/maier/databases/Metaphlan",
                "/mnt/lustre/groups/maier/databases/mOTUs/db_mOTU")) 

dbs_df

# Write file
dbs_file = file.path(sheets_dir, "database_file.csv")
dbs_df %>%
    write_csv(dbs_file)

In [ ]:
# Taxdump folder
taxdump_dir = "/mnt/lustre/groups/maier/databases/Taxdump"

# Host genomes
host_genome = "/mnt/lustre/groups/maier/databases/Host_genomes/hg19_main_mask_ribo_animal_allplant_allfungus.fa"

In [ ]:
# Create command
# Base command
# To run metaphlan add  --run_metaphlan
taxprofiler_cmd = str_glue(
  "conda activate {{conda_env}} && \\
  cd {{out_dir}} && \\
  nextflow run nf-core/taxprofiler -r 1.1.8 \\
  --input {{samples_sheet}} \\
  --databases {{databases_sheet}} \\
  --outdir {{out_dir}} \\
  -profile m3c \\
  --perform_shortread_qc \\
  --perform_shortread_hostremoval \\
  --perform_runmerging \\
  --shortread_qc_dedup \\
  --save_analysis_ready_fastqs \\
  --hostremoval_reference {{host_genome}} \\
  --run_profile_standardisation \\
  --taxpasta_taxonomy_dir {{tax_dir}} \\
  --taxpasta_add_name \\
  --taxpasta_add_rank \\
  --taxpasta_add_lineage \\
  --taxpasta_add_ranklineage \\
  --run_kraken2 \\
  --run_bracken \\
  --run_motus")

In [ ]:
# Fill command
Clean_tax_cmd = str_glue(taxprofiler_cmd,
                         conda_env = conda_env,
                         samples_sheet = samples_sheet_tax,
                         databases_sheet = dbs_file,
                         tax_dir = taxdump_dir,
                         host_genome = host_genome,
                         out_dir = out_dir)

Clean_tax_cmd